
Since we ran this code in kaggle, the file structure follows the kaggle design. And the external files we used are as follows: 
1. Competition Helper code, some Score class, check_submission, etc. \
File path = `../input/helmet-assignment-helpers/helmet-assignment-main/`, can get from the github https://github.com/RobMulla/helmet-assignment 

2. Detection Data. The result from detection step. \
File path = `../input/nfl-health-and-safety-helmet-assignment/test_baseline_helmets.csv`

3. NGS Data. The csv file provided by the competition host. \
File path = `../input/nfl-health-and-safety-helmet-assignment/test_player_tracking.csv`

4. Ground Truth. The csv file provided by the competition host. \
File path = `../input/nfl-health-and-safety-helmet-assignment/train_labels.csv`

4. Video. The video provided by the competition host. \
File path = `../input/nfl-health-and-safety-helmet-assignment/test`

> All the data provided by the competition host can be found here 
https://www.kaggle.com/c/nfl-health-and-safety-helmet-assignment/data

The output file path will be `/kaggle/working/submission.csv`

# Mapping

## Import library and some helper code provide by Kaggle competition

In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

In [ ]:
# Install helmet-assignment helper code
!pip install ../input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.video import video_with_predictions
from IPython.display import Video, display
from helmet_assignment.score import check_submission, NFLAssignmentScorer
from helmet_assignment.features import add_track_features

## Load Detection Data and NGS Data

In [ ]:
debug = False
sample_num = 3
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
#     helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')
    helmets = pd.read_csv('../input/detection/baseline(1).csv')

In [ ]:
helmets.shape

In [ ]:
helmets = helmets.drop_duplicates(subset=["video_frame", "left", "width", "top", "height"], keep='first')

In [ ]:
helmets.shape

In [ ]:
tracking = add_track_features(tracking)

In [ ]:
debug = False
if debug:
    helmets['gameKey'] = helmets['video_frame'].str.split('_').str[0]
    sample_keys = random.sample(list(helmets['gameKey'].unique()), sample_num)
    print(sample_keys)
    tracking = tracking[tracking['gameKey'].astype(str).isin(sample_keys)]
    helmets = helmets[helmets['gameKey'].astype(str).isin(sample_keys)]
    labels = labels[labels['gameKey'].astype(str).isin(sample_keys)]
else:
    helmets['gameKey'] = helmets['video_frame'].str.split('_').str[0]
    sample_keys = list(helmets['gameKey'].unique())
    print(sample_keys)
    tracking = tracking[tracking['gameKey'].astype(str).isin(sample_keys)]
    helmets = helmets[helmets['gameKey'].astype(str).isin(sample_keys)]
    labels = labels[labels['gameKey'].astype(str).isin(sample_keys)]
tracking.shape, helmets.shape, labels.shape

## Mapping main function

In [ ]:
CONF_THRE = 0.3
LEN_DIFF_THRE = 11
EDGE_DIST_THRE = 0.2
RADIUS_THRE = 0.2
NEIGHBOUR_THRE = 1
use_mode_dic = {}

In [ ]:
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def norm_arr(a):
    if len(a) <= 1:
        return [[0, 0]]
    b = a.max(axis=0) - a.min(axis=0)
    for i in range(len(b)):
        if b[i] == 0:
            b[i] = 1
        
    a = (a-a.min(axis=0))/b
    return a

def getCostMatrix(d, t):
    '''
    Args:
        d: nparray of [[x,y]], the center of detection bbox
        t: nparray of [[x,y]], the tracking data
    '''
    assert len(d) <= len(t), f'{len(d)}, {len(t)}'
    ii = len(d)
    jj = len(t)
    cost = np.zeros((ii,jj))
    for i in range(ii):
        for j in range(jj):
            dist = np.linalg.norm(d[i]-t[j])
            cost[i][j] = dist
    
    
    return cost

def getMapIdxAndCost(d, t):
    '''
    Args:
        d: nparray of [[x,y]], the center of detection bbox
        t: nparray of [[x,y]], the tracking data
    '''
    assert len(d) <= len(t), f'{len(d)}, {len(t)}'
    len_diff = len(t) - len(d)
    col_ind = []
    min_cost = None
    min_var = None
    
    cost = getCostMatrix(d, t)
    row_ind, col_ind = linear_sum_assignment(cost)
    min_cost = cost[row_ind, col_ind].sum()
    min_var = np.var(cost[row_ind, col_ind])
            
    return col_ind, min_cost, min_var

def setLabel(detection, tracking_label, col_idx):
    '''
    Args:
        detection: dataframe of one frame detection result
        tracking_label: nparray of player label of one frame
        col_idx: mapping result from KM algorithm
    '''
    tracking_label_reorder = []
    for idx in col_idx:
        tracking_label_reorder.append(tracking_label[idx])
    detection['label'] = tracking_label_reorder

def getSparsePoint(t, n):
    nn = len(t)
    sparse_dic = {}
    for k in range(nn):
        sparse_dic[k] = 0
    for i in range(nn):
        for j in range(i+1, nn):
            dis = np.linalg.norm(t[i]-t[j])
            if dis <= RADIUS_THRE:
                sparse_dic[i] += 1
                sparse_dic[j] += 1
    sparse_dic = sorted(sparse_dic.items(), key = lambda kv:(kv[1], kv[0])) 
    res = []
    for x in sparse_dic:
        if x[1] <= NEIGHBOUR_THRE:
            res.append(x[0])
    if len(res) <= n:
        return [res]
    else:
        del_list = list(itertools.combinations(res,n))
        return del_list
                
def getIdx(arr, n):
    '''
    Args:
        arr: [[idx, distance]]
        n: legth limit
    '''
    res = []
    arr.sort(key=lambda item:item[1])
    for i in arr[:n]:
        res.append(i[0])
    return res
    
    
def getDelList(t, n):
    del_list_lt = []
    del_list_rt = []
    del_list_lb = []
    del_list_rb = []
    del_list_lr = []
    del_list_tb = []
    for j in range(len(t)):
        # left top, x->0, y->1
        if abs(t[j][0] - 0) < EDGE_DIST_THRE or abs(t[j][1]-1)< EDGE_DIST_THRE:
            del_list_lt.append([j, min(abs(t[j][0] - 0), abs(t[j][1]-1))])
        # right top, x->1, y->1
        if abs(t[j][0] - 1) < EDGE_DIST_THRE or abs(t[j][1]-1)< EDGE_DIST_THRE:
            del_list_rt.append([j, min(abs(t[j][0] - 1), abs(t[j][1]-1))])
        # left bottom, x->0, y->0
        if abs(t[j][0] - 0) < EDGE_DIST_THRE or abs(t[j][1]-0)< EDGE_DIST_THRE:
            del_list_lb.append([j, min(abs(t[j][0] - 0), abs(t[j][1]-0))])
        # right bottom, x->1, y->0
        if abs(t[j][0] - 1) < EDGE_DIST_THRE or abs(t[j][1]-0)< EDGE_DIST_THRE:
            del_list_rb.append([j, min(abs(t[j][0] - 1), abs(t[j][1]-0))])
        # left right, x->0, x->1
        if abs(t[j][0] - 0) < EDGE_DIST_THRE or abs(t[j][0]-1)< EDGE_DIST_THRE:
            del_list_lr.append([j, min(abs(t[j][0] - 0), abs(t[j][0]-1))])
        # top bottom, y->1, y->0
        if abs(t[j][1] - 1) < EDGE_DIST_THRE or abs(t[j][1]-0)< EDGE_DIST_THRE:
            del_list_tb.append([j, min(abs(t[j][1] - 1), abs(t[j][1]-0))])
    
    return [getIdx(del_list_lt,n), getIdx(del_list_rt,n), 
            getIdx(del_list_lb,n), getIdx(del_list_rb,n), 
            getIdx(del_list_lr,n), getIdx(del_list_tb,n)]
    
def KMmain(detection, tracking, view, use_mode):
    '''
    Args:
        detection: dataframe of one frame detection result
        tracking: dataframe of one frame tracking data
    
    '''
    # detection result number must <= tracking data number
    assert len(detection) <= len(tracking), f'{len(detection)}, {len(tracking)}'

    # get length diff
    len_diff = len(tracking) - len(detection)
    
    detection_data_p = None
    detection_data_n = None
    if view == 'Endzone':
        if use_mode == 1:
            detection_data_p = detection[['center_h_p','center_v_p']].to_numpy()
        elif use_mode == -1:
            detection_data_n = detection[['center_h_n','center_v_n']].to_numpy()
        else:
            detection_data_p = detection[['center_h_p','center_v_p']].to_numpy()
            detection_data_n = detection[['center_h_n','center_v_n']].to_numpy()
    else:
        if use_mode == 1:
            detection_data_p = detection[['center_h_p','center_v_n']].to_numpy()
        elif use_mode == -1:
            detection_data_n = detection[['center_h_n','center_v_p']].to_numpy()
        else:
            detection_data_p = detection[['center_h_p','center_v_n']].to_numpy()
            detection_data_n = detection[['center_h_n','center_v_p']].to_numpy()
        
    tracking_data= tracking[['x','y']].to_numpy()

    # min-max nomalization
    if use_mode == 1:
        detection_data_p = norm_arr(detection_data_p)
    elif use_mode == -1:
        detection_data_n = norm_arr(detection_data_n)
    else:
        detection_data_p = norm_arr(detection_data_p)
        detection_data_n = norm_arr(detection_data_n)
    tracking_data = norm_arr(tracking_data)
    
    # store data for later comparison
    col_idx_list_p = []
    cost_list_p = []
    var_list_p = []
    
    col_idx_list_n = []
    cost_list_n = []
    var_list_n = []
    
    if use_mode == 1:
        this_col_idx, this_cost, this_var = getMapIdxAndCost(detection_data_p, tracking_data)
        
        col_idx_list_p.append(this_col_idx)
        cost_list_p.append(this_cost)
        var_list_p.append(this_var)
    elif use_mode == -1:
        this_col_idx_n, this_cost_n, this_var_n = getMapIdxAndCost(detection_data_n, tracking_data)
        
        col_idx_list_n.append(this_col_idx_n)
        cost_list_n.append(this_cost_n)
        var_list_n.append(this_var_n)
    else:
        this_col_idx, this_cost, this_var = getMapIdxAndCost(detection_data_p, tracking_data)
        this_col_idx_n, this_cost_n, this_var_n = getMapIdxAndCost(detection_data_n, tracking_data)
        
        col_idx_list_p.append(this_col_idx)
        cost_list_p.append(this_cost)
        var_list_p.append(this_var)

        col_idx_list_n.append(this_col_idx_n)
        cost_list_n.append(this_cost_n)
        var_list_n.append(this_var_n)

    delete_idx_list = [[]]
    
    if len_diff != 0 and len_diff <= LEN_DIFF_THRE:
        del_list = getSparsePoint(tracking_data, len_diff) + getDelList(tracking_data, len_diff)

        for delete_idx in del_list:
            this_tracking = np.delete(tracking_data, delete_idx, axis=0)
            this_tracking = norm_arr(this_tracking)
            
            if use_mode == 1:
                this_col_idx, this_cost, this_var = getMapIdxAndCost(detection_data_p, this_tracking)
                
                col_idx_list_p.append(this_col_idx)
                cost_list_p.append(this_cost)
                var_list_p.append(this_var)
            elif use_mode == -1:
                this_col_idx_n, this_cost_n, this_var_n = getMapIdxAndCost(detection_data_n, this_tracking)
                
                col_idx_list_n.append(this_col_idx_n)
                cost_list_n.append(this_cost_n)
                var_list_n.append(this_var_n)
            else: 
                this_col_idx, this_cost, this_var = getMapIdxAndCost(detection_data_p, this_tracking)
                this_col_idx_n, this_cost_n, this_var_n = getMapIdxAndCost(detection_data_n, this_tracking)

                col_idx_list_p.append(this_col_idx)
                cost_list_p.append(this_cost)
                var_list_p.append(this_var)

                col_idx_list_n.append(this_col_idx_n)
                cost_list_n.append(this_cost_n)
                var_list_n.append(this_var_n)
            
            delete_idx_list.append(delete_idx)
   
    
    var_all = var_list_p + var_list_n
    col_idx_all = col_idx_list_p + col_idx_list_n
    delete_idx_all = delete_idx_list + delete_idx_list

    min_idx = np.argmin(var_all)

    min_var = var_all[min_idx]
    min_delete_idx = delete_idx_all[min_idx]
    min_col_idx = col_idx_all[min_idx]

    
    tracking_player = np.delete(tracking['player'].to_numpy(), min_delete_idx)
    setLabel(detection, tracking_player, min_col_idx)
    
    out_use_mode = 0
    # check use p or n
    if use_mode == 0:
        if min_idx < len(var_all) / 2:
            out_use_mode = 1
        else:
            out_use_mode = -1
    else:
        out_use_mode = use_mode
        
    detection['var'] = min_var
    detection['len_diff'] = len_diff
    return min_var, len_diff, out_use_mode

# main function
def mapping_df(dff, use_mode = 0):
    '''
    Args:
        video_frame: the name of video+frame
        df: dataframe, all bbox(left, width, top, height, conf) of this frame
    '''
    video_frame, df = dff
    # get these from video_frame name
    gameKey,playID,view,frame = video_frame.split('_')
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)

    # find tracking data for this play
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    # get nearest frame number in tracking data
    # with different frame number, can return same est_frame
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    # get all data with the frame number
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
    # the len should = 22, each play has 22 players
    len_this_tracking = len(this_tracking)

    # get bbox center value
    # _h means horizontal, _v means vertical
    # _p means positive, _m means minus
    df['center_h_p'] = (df['left']+df['width']/2)
    df['center_v_p'] = (df['top']+df['height']/2)
    df['center_h_n'] = (df['left']+df['width']/2)*-1
    df['center_v_n'] = (df['top']+df['height']/2)*-1
    # get bbox conf > 0.3
    df = df[df['conf']>CONF_THRE].copy()
    # df only choose the first 22 value
    # from our statistic, len(df) always lower than 22, means we often lose some bbox
    if len(df) > len_this_tracking:
        df = df.tail(len_this_tracking)
    
    # if the view is endzone, swap x, y for the tracking data
    if view == 'Endzone':
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
    
    if frame < 10:
        use_mode = 0
    else:
        first_video_frame_of_this_video = '_'.join(video_frame.split('_')[:3]) + '_' + '1'
        if first_video_frame_of_this_video in use_mode_dic.keys():
            use_mode = use_mode_dic[first_video_frame_of_this_video]
        
    min_var, len_diff, out_use_mode = KMmain(df, this_tracking, view, use_mode)
    if frame < 10:
        use_mode_dic[video_frame] = out_use_mode
    
    if view_graph:
        print('video_frame = {}, len_diff = {}, min_var = {}, use_mode = {}'.format(video_frame, len_diff, min_var, out_use_mode))
        return df[['video_frame','left','width','top','height','label', 'var', 'len_diff']], df[['center_h_p', 'center_v_p', 'label']], this_tracking[['x','y', 'player']], video_frame
    else:
#         if len_diff > 10:
#             print('video_frame = {}, len_diff = {}, min_var = {}, use_mode = {}'.format(video_frame, len_diff, min_var, out_use_mode))
        return df[['video_frame','left','width','top','height','label', 'var', 'len_diff']]

In [ ]:
def visulizeResult(detection, tracking, gt):
    plt.figure(figsize=(10, 10))
    a = detection.to_numpy()
    aa = norm_arr(a[:, 0:2]*-1)
    b = tracking.to_numpy()
    bb = norm_arr(b[:, 0:2])
    gt['center_h_p'] = (gt['left']+gt['width']/2)
    gt['center_v_p'] = (gt['top']+gt['height']/2)
    c = gt[['center_h_p', 'center_v_p', 'label']].to_numpy()
    cc = norm_arr(c[:, 0:2]*-1)
    aa = np.array(aa)
    plt.scatter(aa[:,0], aa[:,1], marker = '^', color = 'red', label = 'Detection')
    plt.scatter(bb[:,0], bb[:,1], marker = 'o', color = 'green', label = 'Tracking')
    plt.scatter(cc[:,0], cc[:,1], marker = 'x', color = 'blue', label = 'Truth')
    for i in range(len(a)):
        for j in range(len(b)):
            if a[i][2] == b[j][2]:
                plt.plot([aa[i][0], bb[j][0]], [aa[i][1], bb[j][1]], color='k', linestyle='--')
                
    for i in range(len(c)):
        for j in range(len(b)):
            if c[i][2] == b[j][2]:
                plt.plot([cc[i][0], bb[j][0]], [cc[i][1], bb[j][1]], linestyle='--', color='orange')
    
    plt.legend()
    plt.show()

## view result for one frame

In [ ]:
df_list = list(helmets.groupby('video_frame'))
view_graph = True
use_mode = -1
frame_to_test = '57995_000109_Endzone_160'
for i in df_list:
    x, y = i
    gameKey,playID,view,frame = x.split('_')
    if x != frame_to_test:
        continue
        
    res, a, b, _ = mapping_df(dff=i, use_mode=use_mode)
    gt = labels[labels['video_frame'] == x]
    visulizeResult(a, b, gt[['height', 'left', 'width', 'top','label']])
    print('res ============')
    print(res)

## Do mapping for each frame

In [ ]:
df_list = list(helmets.groupby('video_frame'))
view_graph = True
submission_df_list = []
with tqdm(total=len(df_list)) as pbar:
    for i in df_list:
        x, y = i
        if view_graph:
            res, a, b, _ = mapping_df(dff=i)
            submission_df_list.append(res)
            gt = labels[labels['video_frame'] == x]
            visulizeResult(a, b, gt[['height', 'left', 'width', 'top','label']])
        else:
            res = mapping_df(dff=i)
            submission_df_list.append(res)
        pbar.update(1)

In [ ]:
submission_df = pd.concat(submission_df_list)

In [ ]:
output = submission_df[['video_frame', 'left', 'width', 'top', 'height', 'label']]

In [ ]:
scorer = NFLAssignmentScorer(labels, impact_weight=1)
# print(scorer.check_submission(output))
baseline_score = scorer.score(output)
print(f"validation score {baseline_score:0.4f}")

In [ ]:
mapping_result = submission_df 

The output should be name as 'mapping_result' then go to deepsort